In [1]:
import cv2
import numpy as np
from keras.models import model_from_json 

In [2]:
emotion_dict = {0:"Angry", 1:"Digusted", 2:"Fearful", 3:"Happy", 4: "Neutral", 
               5:"Sad", 6:"Supterised"}

#load json and create models
json_file = open("emotion_model.json", "r")
loaded_model_json = json_file.read()
json_file.close()

emotion_model = model_from_json(loaded_model_json)
print("Loaded model from disk.................")

Loaded model from disk.................


In [3]:
#Start the webcam feed to test the emotion model

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    #frame = cv2.resize(frame,(1280,720))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #Detects the faces in the camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor = 1.3,
                                              minNeighbors=5)
    
    for (x,y,w,h) in num_faces:
        cv2.rectangle(frame, (x,y-50), (x+w, y+h+10), (0,255,0), 4)
        roi_gray_frame = gray_frame[y:y+h, x:x+w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48,48)),-1),0) #(48,48) must be same as the training model for target size.
        
        #predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img)
        maxIndex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxIndex], (x+5,y-20), cv2.FONT_HERSHEY_COMPLEX, 1,(0,255,0), 2, cv2.LINE_AA)
        
    cv2.imshow("Emotion Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 19ms/step
